<a href="https://colab.research.google.com/github/LKSfiuza/Machine-Learning-projects/blob/main/Adicionando_novas_Features_Kaggle_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Titanic - Machine Learning from Disaster

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

In [80]:
# Abrir o dataset de treino e teste

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [81]:
# Pré-processamento dos dados

# Descrição estatistica das fetures numericas

train.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [82]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [83]:
# Verificando valores nulos ou NaN

print(train.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [84]:
print(test.isnull().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [85]:
# Criando os dados de treinamento e teste

# Retirada das colunas ID e Survived

X_train = train.drop(['PassengerId', 'Survived'], axis = 1)

X_test = test.drop(['PassengerId'], axis = 1)

In [87]:
# Modificando as coluna Sex para valores de 1 para Female e 0 para Male
def criar_features(X):
  subs = {'male': 0, 'female': 1}
  X['mulher'] = X['Sex'].replace(subs)

  X['Age'] = X['Age'].fillna(X['Age'].mean())

  X['Fare'] = X['Fare'].fillna(X['Fare'].mean())

  X['Embarked'] = X['Embarked'].fillna('S')

  subs = {'S': 0, 'C': 1 , 'Q' :3}
  X['porto'] = X['Embarked'].replace(subs)

  X['crianca'] = 1
  X['crianca'] = np.where(X['Age'] < 12 , 1, 0)

  return X
X_train = criar_features(X_train)
X_test = criar_features(X_test)

In [88]:
X_test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,mulher,porto,crianca
0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,3,0
1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1,0,0
2,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,3,0
3,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,0,0
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1,0,0


In [89]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [90]:
features = ['Pclass','mulher', 'Age','Parch', 'Fare', 'porto','SibSp','crianca']

In [91]:
# Selecionando as features

X_train = X_train[features]
X_test = X_test[features]

y_train = train['Survived']

In [92]:
# Padronização das variaveis

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [93]:
# Modelo e validação cruzada

model_lr = LogisticRegression(max_iter=1000, random_state=0)

score = cross_val_score(model_lr, X_train, y_train, cv = 10)

print(score.mean())

0.8035955056179775


In [94]:
# Modelo final

model_lr.fit(X_train, y_train)

y_pred = model_lr.predict(X_train)

In [95]:
mc = confusion_matrix(y_train, y_pred)
print(mc)

[[479  70]
 [ 99 243]]


In [96]:
score = model_lr.score(X_train, y_train)
print(score)

0.8103254769921436


In [97]:
# Predição nos dados de test

y_pred = model_lr.predict(X_test)

In [98]:
submissao = pd.DataFrame(test['PassengerId'])
submissao['Survived'] = y_pred

In [99]:
submissao.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [100]:
submissao.to_csv('submissao3.csv', index = False)